### Notes

* interactions.json `C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\interactions\interactions.json`
    * events
    * choices
    * new socials
    * item loot
* loot.json `C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\loot\loot.json`
    * item loot?
    * condition loot (age, skills, traits)
* conditions.json `C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\conditions\conditions.json`
    * friendly names of conditions
* lifeevents.json `C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\lifeevents\lifeevents.json`
    * money

#### TODO
* Maybe schemaify to objects because [""] is fun to type
* Maybe some string constants
* Maybe use whatever localization/friendly names exist
* Maybe separate the Seek Adventure, Save Some Money, and Seek Ship events

### File reads

In [13]:
interactionsFilePath = r"C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\interactions\interactions.json"
lootFilePath = r"C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\loot\loot.json"
lifeeventsFilePath = r"C:\Games\steamapps\common\Ostranauts\Ostranauts_Data\StreamingAssets\data\lifeevents\lifeevents.json"

with open(interactionsFilePath) as f:
    intRaw = f.read()

with open(lootFilePath) as f:
    lootRaw = f.read()

with open(lifeeventsFilePath) as f:
    lifeRaw = f.read()

### Get the entries we care about

In [14]:
import msgspec

jsonInteractions = msgspec.json.decode(intRaw)
jsonLoot = msgspec.json.decode(lootRaw)
jsonLife = msgspec.json.decode(lifeRaw)

relInts = {}
for i in jsonInteractions:
    if i["strName"].startswith("CG"):
        relInts[i["strName"]] = i
relInts = dict(sorted(relInts.items()))

loot = {}
for i in jsonLoot:
    loot[i["strName"]] = i

life = {}
for i in jsonLife:
    life[i["strInteraction"]] = i

len(relInts)

91

### Create a set containing all choices in all interactions

In [15]:
allChoices = set()

for i in relInts:
    if "aInverse" in relInts[i]:
        for j in relInts[i]["aInverse"]:
            allChoices.add(j)

len(allChoices)

70

### Get all the data

In [16]:
entries = [f'']

for i in relInts:
	if not "strTitle" in relInts[i]:
		continue
	if not "strName" in relInts[i]:
		continue
	if relInts[i]["strName"] in allChoices:
		continue
	if not "aInverse" in relInts[i]:
		continue
	
	name = relInts[i]["strName"]	# code name
	title = relInts[i]["strTitle"]	# friendly name
	body = relInts[i]["strDesc"]
	rawChoices = relInts[i]["aInverse"]
	intLoots = f''
	intMoney= f''

	if name in life:
		if "fCashRewardMax" in life[name]:
			intMoney += f'\n${life[name]["fCashRewardMin"]:,.2f} to ${life[name]["fCashRewardMax"]:,.2f}'
		elif not life[name]["fCashRewardMin"] == 0:
			intMoney += f'\n${life[name]["fCashRewardMin"]:,.2f}'
	
	if name in loot:
		t = loot[name]
		if t["aCOs"]:
			for a in t["aCOs"]:
				intLoots += f'\nCondition: `{a}`'
		if t["aLoots"]:
			for l in t["aLoots"]:
				intLoots += f'\nItem/Other: `{l}`'

	choices = ""
	for c in rawChoices:
		if not c in relInts:
			continue
		cData = relInts[c]
		choiceTitle = cData["strTitle"]

		rewards = f''
		if "aLootItms" in cData and cData["aLootItms"]:
			items = f''
			if cData["aLootItms"][0].startswith("addus,"):
				lootItm = cData["aLootItms"][0].removeprefix("addus,")
				isEmpty = True
				if lootItm in loot:
					t = loot[lootItm]
					if t["aCOs"]:
						for a in t["aCOs"]:
							items += f'\n\t* Condition: `{a}`'
							isEmpty = False
					if t["aLoots"]:
						for l in t["aLoots"]:
							items += f'\n\t* Item/Other: `{l}`'
							isEmpty = False
				if isEmpty:
					items += f'\n\t* Item/Other: `{lootItm}`'
			rewards += items

		socials = f''
		if "aSocialNew" in cData and cData["aSocialNew"]:
			for s in cData["aSocialNew"]:
				if not s:
					continue
				socials += f'\n\t* Social: `{s}`'

		loots = f''
		if "LootCTsUs" in cData:
			if cData["LootCTsUs"] in loot:
				thisLoot = loot[cData["LootCTsUs"]]
				for cond in thisLoot["aCOs"]:
					loots += f'\n\t* Condition: `{cond}`'

				for stuff in thisLoot["aLoots"]:
					loots += f'\n\t* Item/Other: `{stuff}`'
			if not loots:
				loots += f'\n\t* {cData["LootCTsUs"]}'
		
		conds = f''
		if "LootCondsUs" in cData:
			if cData["LootCTsUs"] in loot:
				thisLoot = loot[cData["LootCondsUs"]]
				for cond in thisLoot["aCOs"]:
					conds += f'\n\t* Condition: `{cond}`'

				for stuff in thisLoot["aLoots"]:
					conds += f'\n\t* Item/Other: `{stuff}`'
			if not conds:
				conds += f'\n\t* {cData["LootCondsUs"]}'

		money = f''
		shipReward = f''
		if cData["strName"] in life:
			lifeData = life[cData["strName"]]

			if "fCashRewardMax" in lifeData and not lifeData["fCashRewardMax"] == 0.0:
				money += f'\n\t* ${lifeData["fCashRewardMin"]:,.2f} to ${lifeData["fCashRewardMax"]:,.2f}'
			elif not lifeData["fCashRewardMin"] == 0.0:
				money += f'\n\t* ${lifeData["fCashRewardMin"]:,.2f}'

			if "strShipRewards" in lifeData and lifeData["strShipRewards"]:
				shipReward += f'\n\t* Ship: `{lifeData["strShipRewards"]}`'
		

		rewards += socials
		rewards += loots
		rewards += conds
		rewards += money
		rewards += shipReward
		rewards += "\n"


		choices += f'''\n* **{choiceTitle}** `{cData["strName"]}`
{rewards}'''

	if not intMoney and not intLoots and not choices:
		continue
	entries.append(f'''\n### **{title}** `{name}`{intMoney}{intLoots}
{choices}''')

output = "# **Ostranauts Career Dump**"+ "\n".join(entries)

### Print markdown to file

In [17]:
with open("CareerDump.md", "w") as f:
    f.write(output)

import shutil

shutil.copyfile("CareerDump.md", "README.md")

### Poop it out here

In [18]:
from IPython.display import display_markdown

display_markdown(output, raw=True)

# **Ostranauts Career Dump**

### **Crackdown** `CGEncCrackdownIntro`

* **Hide In An Air Vent** `CGEncCrackdownVents`

	* Social: `CGEncRandomAdultFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillStealth=1.0x1`
	* Condition: `TUpSkillIntrusion=1.0x1`
	* Condition: `TUpIsCraven=1.0x1`
	* $246.00

* **Bluff** `CGEncCrackdownBluff`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillForgery=1.0x1`
	* Condition: `TUpSkillActing=1.0x1`
	* $500.00

* **Have Nothing To Hide** `CGEncCrackdownNothingToHide`

	* Social: `CGEncHomeworldRandomLEOFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteStreet=1.0x1`
	* Condition: `TUpIsHonest=1.0x1`
	* Condition: `TUpIsArrogant=1.0x1`

* **Fake A Pressure Alarm** `CGEncCrackdownFakeAPressureAlarm`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillHacking=1.0x1`
	* Condition: `TUpIsGenius=1.0x1.0`

* **RISK: Be A Hero (Unarmed Melee)** `CGEncCrackdownBeAHeroSuccess`

	* Social: `CGEncHomeworldLEOEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillRangedArchery=1.0x1`
	* Condition: `TUpIsBrave=1.0x1`
	* Condition: `TDnEncHeroFailChance=1.0x1`
	* Item/Other: `CTEncRandomInjuryPastFight=1.0x1`
	* $1,750.00

* **RISK: Be A Hero (Unarmed Melee)** `CGEncCrackdownBeAHeroFail`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsImprisoned=1.0x1`
	* Condition: `TDnEncHeroFailChance=1.0x1`


### **Daily Grind** `CGEncDailyGrindIntro`

* **Hacking** `CGEncDailyGrindHacking`

	* Condition: `ItmComponentMobo01=1.0x1`
	* Condition: `ItmHeatSink01=1.0x1`
	* Social: `CGEncHomeworldLEOEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillHacking=1.0x1`

* **Watch Old Movies** `CGEncDailyGrindOldMovies`

	* Social: `CGEncHomeworldRandomManagerFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsObservant=1.0x1.0`
	* Condition: `TUpSkillActing=1.0x1`

* **Suit Maintenance** `CGEncDailyGrindFeverishSuitMaintenance`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillOpsEnvSuit=1.0x1`
	* Condition: `TUpSkillSciencePhysics=1.0x1`
	* Condition: `TUpIsPatient=1.0x1`
	* Condition: `TUpIsFinicky=1.0x1`

* **Fight Club** `CGEncDailyGrindUndergroundBoxing`

	* Item/Other: `COSocialNameDropLEO`
	* Social: `CGEncHomeworldRandomLEOFriend`
	* Condition: `TUpSkillMeleeUnarmed=1.0x1`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsBrave=1.0x1`
	* Condition: `TUpIsMasochist=1.0x1`
	* Item/Other: `CTEncRandomInjuryPastFight=1.0x1`

* **Go Shipspotting** `CGEncDailyGrindShippingNews`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsComplacent=1.0x1`
	* Condition: `TUpSkillOpsSensors=1.0x1`
	* Condition: `TUpIsShy=1.0x1.0`
	* $650.00

* **Get a Second Job as an Electrician** `CGEncDailyGrindKioskJob`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsPatient=1.0x1.0`
	* Condition: `TUpSkillEngElectronic=1.0x1`
	* $200.00


### **Manslaughter.** `CGEncManslaughter`

* **Time in the Brig** `CGEncImprisoned`

	* Social: `CGEncHomeworldLEOEnemy`
	* Social: `CGEncHomeworldPrisonerFriend`
	* Social: `CGEncHomeworldCriminalEnemy`
	* Social: `CGEncHomeworldCriminalContact`
	* Condition: `TUpAge=1.0x1-3`
	* Condition: `TDnIsImprisoned=1.0x1`
	* Item/Other: `CTEncRandomInjuryPastFight=1.0x1-3`
	* Condition: `CareerPrisonerPast=1.0x1`
	* Condition: `TraitChronicScarCuffs=0.9x1`
	* Condition: `SkillMeleeUnarmed=0.7x1|IsQuitter=0.1x1|IsFollower=0.1x1`
	* Condition: `IsCraven=0.2x1|IsBrave=0.2x1|IsSuspicious=0.2x1|IsPushover=0.1x1`
	* Condition: `IsApathetic=0.2x1|TraitChronicScarActivism=0.15x1`
	* Condition: `SkillStealth=0.2x1|TraitChronicScarBrandThief=0.15x1`
	* Condition: `IsObservant=0.2x1|TraitChronicScarIndustrial=0.15x1`
	* Condition: `IsPatient=0.2x1|IsImpatient=0.2x1`
	* Condition: `IsTreacherous=0.1x1|IsSelfish=0.1x1|IsVengeful=0.1x1|IsAgliophobic=0.1x1`
	* Item/Other: `CONDAgingChanceRandomChronicCondition=0.5x1-3`


### **Occupational Hazard** `CGEncPodDisasterIntro`

* **Hack Engine** `CGEncPodDisasterHackEngine`

	* Social: `CGEncCriminalContact`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillHacking=1.0x1`
	* Condition: `TUpIsGenius=1.0x1.0`

* **Take the Stick** `CGEncPodDisasterNavigate`

	* Social: `CGEncRandomAdultFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillOpsSpaceship=1.0x1`
	* Condition: `TUpSkillOpsEnvSuit=1.0x1`
	* Item/Other: `CTEncRandomInjuryLeg=1.0x1`

* **Jury Rig Engine** `CGEncPodDisasterEngMechanical`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEngMechanical=1.0x1`
	* Condition: `TUpIsBrave=1.0x1`
	* Condition: `TUpIsCoordinated=1.0x1.0`
	* $-150.00

* **Troubleshoot Engine** `CGEncPodDisasterEngSpaceship`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillOpsSpaceship=1.0x1`
	* $850.00

* **RISK: Abandon ship! (EVA Suit Ops)** `CGEncPodDisasterRiskAbandonShipSuccess`

	* Item/Other: `OutfitEVA01Full=1.0x1`
	* Item/Other: `OutfitHelmet03=1.0x1`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TDnEncPodDisasterFailChance=1.0x1`

* **RISK: Abandon ship! (EVA Suit Ops)** `CGEncPodDisasterRiskAbandonShipFail`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TDnEncPodDisasterFailChance=1.0x1`
	* Item/Other: `CTEncRandomInjuryBurn=1.0x1`
	* $-1,050.00


### **Poker Game** `CGEncPokerGameIntro`

* **RISK: Call The Bluff (Gambling Skill)** `CGEncPokerGameCallBluffSuccess`

	* Item/Other: `ItmShipbreakerLoadout`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillGambling=1.0x1`
	* Condition: `TDnEncPokerFailChance=1.0x1`
	* Ship: `CGEncShipbreakerPrizeShip`

* **RISK: Call The Bluff (Gambling Skill)** `CGEncPokerGameCallBluffFail`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillGambling=1.0x1`
	* Condition: `TDnEncPokerFailChance=1.0x1`
	* $-9,000.00

* **No Thanks...Fold** `CGEncPokerGameFold`

	* 
	* $900.00

* **Cheat** `CGEncPokerGameCheat`

	* Social: `CGEncHomeworldLEOEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillHacking=1.0x1`
	* Item/Other: `CTEncRandomInjuryPastFight=1.0x1`
	* $1,500.00

* **Grab The Key And Run** `CGEncPokerGameYoink`

	* Item/Other: `ItmShipbreakerLoadout`
	* Social: `CGEncCriminalEnemy`
	* Social: `CGEncRandomEnemy`
	* Social: `CGEncRandomEnemy`
	* Condition: `TUpAge=1.0x1`
	* $-6,500.00
	* Ship: `CGEncShipbreakerPrizeShip`

* **Call Him On A Technicality** `CGEncPokerGameTechnicality`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillGambling=1.0x1`
	* $300.00


### **Rare Salvage** `CGEncRareSalvageIntro`

* **Notify Ayotimiwa Corporate** `CGEncRareSalvageTellAyotimiwa`

	* Condition: `OutfitTShirt05=1.0x1`
	* Social: `CGEncHomeworldManagerContact`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillAdmin=1.0x1`
	* Condition: `TUpIsFollower=1.0x1`

* **Smuggle It Through the Blackmarket** `CGEncRareSalvageSmuggleItOffSite`

	* Item/Other: `COSocialNameDropCrim`
	* Social: `CGEncHomeworldCriminalContact`
	* Social: `CGEncHomeworldLEOEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteStreet=1.0x1.0`
	* $200.00

* **Bribe Your Boss and Split the Profits** `CGEncRareSalvageBribeBoss`

	* Item/Other: `COSocialNameDropManager`
	* Social: `CGEncHomeworldLEOEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteOligarch=1.0x1.0`
	* $750.00

* **RISK: Take It For Yourself (Patient)** `CGEncRareSalvageTakeItForYourselfSuccess`

	* Condition: `ItmRCSDistro01Loose=1.0x1`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TDnEncStealFailChance=1.0x1`

* **Smash It and Feel Alive** `CGEncRareSalvageSmash`

	* Item/Other: `ItmHydraComponents=1.0x1`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillMeleeArmed=1.0x1.0`
	* Condition: `TUpIsCruel=1.0x1.0`
	* Condition: `TUpIsPessimist=1.0x1`
	* Condition: `TUpIsStrong=1.0x1`
	* Condition: `TUpTraitChronicScarCargo=1.0x1`

* **RISK: Take It For Yourself (Patient)** `CGEncRareSalvageTakeItForYourselfFail`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsImprisoned=1.0x1`
	* Condition: `TDnEncStealFailChance=1.0x1`


### **King Incognito** `CGEncRichSchmuckIntro`

* **Start a Riot** `CGEncRichSchmuckInciteRiot`

	* Social: `CGEncHomeworldBartenderEnemy`
	* Social: `CGEncHomeworldManagerEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteFolk=1.0x1`
	* Condition: `TUpSkillMeleeUnarmed=1.0x1`
	* Condition: `TUpSkillPersuade=1.0x1`
	* Condition: `TUpIsCharismatic=1.0x1`
	* Condition: `TUpIsLeader=1.0x1`
	* Item/Other: `CTEncRandomInjuryPastFight=1.0x1`
	* $246.00

* **Pick their Pocket** `CGEncRichSchmuckPickpocket`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillStealth=1.0x1`
	* Condition: `TUpSkillTracking=0.25x1.0`
	* Condition: `TUpIsObservant=0.5x1.0`
	* Condition: `TUpIsPatient=0.5x1.0`
	* Condition: `TUpIsSelfish=0.5x1.0`
	* $250.00

* **Just Mug Them** `CGEncRichSchmuckMug`

	* Social: `CGEncHomeworldManagerEnemy`
	* Social: `CGEncHomeworldBartenderEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillMeleeArmed=0.65x1.0`
	* Condition: `TUpSkillMeleeUnarmed=0.65x1.0`
	* Condition: `TUpSkillRangedGunpowder=0.65x1.0`
	* Condition: `TUpIsBrave=0.2x1.0`
	* Condition: `TUpIsCruel=0.2x1.0`
	* Condition: `TUpIsImpatient=0.2x1.0`
	* Condition: `TUpIsStrong=1.0x1`
	* $246.00

* **Seduce Them** `CGEncRichSchmuckSeduce`

	* Item/Other: `COSocialNameDropManager`
	* Social: `CGEncHomeworldManagerLoverEx`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteOligarch=0.4x1.0`
	* Condition: `TUpSkillCooking=1.0x1.0`
	* Condition: `TUpSkillDance=1.0x1.0`
	* Condition: `TUpIsCharismatic=1.0x1.0`
	* Condition: `TUpIsBeautiful=1.0x1.0`

* **Get them Drunk** `CGEncRichSchmuckCorpSecrets`

	* Social: `CGEncHomeworldManagerContact`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillSong=1.0x1`
	* Condition: `TUpIsLiar=1.0x1`
	* Condition: `TUpIsSlovenly=1.0x1`
	* $850.00

* **RISK: Hack Them (Hacking)** `CGEncRichSchmuckHackPhone`

	* Social: `CGEncHomeworldManagerEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEngSoftware=1.0x1.0`
	* Condition: `TUpSkillEtiquetteStreet=0.35x1.0`
	* Condition: `TUpSkillHacking=0.5x1.0`
	* Condition: `TUpSkillStealth=0.35x1.0`
	* Condition: `TUpIsAmbitious=0.4x1.0`
	* Condition: `TUpIsDiligent=0.4x1.0`
	* Condition: `TUpIsGenius=0.4x1.0`
	* Condition: `TUpIsObservant=0.4x1.0`
	* Condition: `TUpIsShy=0.4x1.0`
	* Condition: `TDnEncRichHackFailChance=1.0x1`
	* $2,850.00

* **RISK: Hack Them (Hacking)** `CGEncRichSchmuckHackPhoneFail`

	* Social: `CGEncHomeworldManagerEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsAmbitious=1.0x1.0`
	* Condition: `TUpIsBrave=1.0x1`
	* Condition: `TDnEncRichHackFailChance=1.0x1`
	* $2,850.00


### **Shady deal.** `CGEncShadyDealIntro`

* **RISK: Join the Scheme (Street Smarts)** `CGEncShadyDealJoinFail`

	* Social: `CGEncRandomEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsImprisoned=1.0x1`
	* Condition: `TDnEncShadyDealFailChance=1.0x1`

* **Gather Intel for Cops** `CGEncShadyDealGatherIntel`

	* Social: `CGEncHomeworldRandomLEOFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteStreet=1.0x1`
	* Condition: `TUpIsHonest=1.0x1`

* **RISK: Join the Scheme (Street Smarts)** `CGEncShadyDealJoinSuccess`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TDnEncShadyDealFailChance=1.0x1`
	* $4,500.00

* **Backstab Them** `CGEncShadyDealBackstab`

	* Social: `CGEncCriminalEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsTreacherous=1.0x1`
	* $900.00

* **Report to Ayotimiwa** `CGEncShadyDealReportThem`

	* Social: `CGEncHomeworldManagerContact`
	* Social: `CGEncHomeworldRandomLEOFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpIsSelfish=1.0x1.0`
	* Item/Other: `ItmShirtHoodie01=1.0x1`
	* $900.00

* **Instigate Violence** `CGEncShadyDealViolence`

	* Social: `CGEncCriminalEnemy`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillMeleeArmed=1.0x1.0`
	* Condition: `TUpSkillMeleeUnarmed=1.0x1.0`
	* Condition: `TUpIsTough=1.0x1`
	* Item/Other: `CTEncRandomInjuryPastFight=1.0x1`
	* $900.00


### **Hulk Salvage.** `CGEncShipHulkSalvage`
$10,000.00


### **Makeshift Salvage Pod.** `CGEncShipMakeshiftSalvagePodIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvageSledTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerSled`


### **Hulk Salvage.** `CGEncShipPAX2020Intro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipPAX2020Take`

	* Ship: `CGEncShipbreakerPAX2020Ships`


### **Retrofitted Shipping Container.** `CGEncShipSalvageContainerPodIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvageContainerPodTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerContainerPod`


### **Endurance Salvage Pod.** `CGEncShipSalvagePodEnduranceIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvagePodEnduranceTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerPodEndurance`


### **Salvage Pod.** `CGEncShipSalvagePodIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvagePodTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* CGEncShipSalvagePodTake
	* Ship: `CGEncShipbreakerPod`


### **Small Salvage Pod.** `CGEncShipSalvagePodSmallIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvagePodSmallTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerPodSmall`


### **Salvage Pod.** `CGEncShipSalvagePodTestIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvagePodTestTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerPodTest`


### **Utility Pod.** `CGEncShipSalvageUtilityPodIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipSalvageUtilityPodTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerUtilityPod`


### **Small Shuttle.** `CGEncShipShuttleSmallIntro`

* **Continue Career** `CGEncShipSalvagePodCont`

	* Condition: `TUpAge=1.0x1`

* **Take Ship** `CGEncShipShuttleSmallTake`

	* Item/Other: `ItmShipbreakerLoadout`
	* Ship: `CGEncShipbreakerShuttleSmall`


### **Save Some Money.** `CGEncShipbreakerSaveMoney`
$400.00 to $600.00
Condition: `TUpAge=1.0x1`


### **Labor Barge Locker** `CGEncTestIntro`

* **RISK: Force the Lock (Strong)** `CGEncTestA`

	* Social: `CGEncShipbreakerFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TDnEncLockFailChance=1.0x1`
	* Item/Other: `ItmToolLaserTorch01New=1.0x1`
	* $-200.00

* **RISK: Force the Lock (Strong)** `CGEncTestAFail`

	* Condition: `TUpAge=1.0x1`
	* Condition: `TDnEncLockFailChance=1.0x1`
	* $-2,300.00

* **Hack the Lock** `CGEncTestB`

	* Social: `CGEncShipbreakerFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillHacking=1.0x1`
	* Condition: `TUpIsLeader=1.0x1`

* **Cut Hole in Crate** `CGEncTestC`

	* Condition: `TUpAge=1.0x1`
	* Item/Other: `CTEncRandomInjuryBurn=1.0x1`
	* $700.00

* **Mislabel Crate** `CGEncTestD`

	* Social: `CGEncShipbreakerFriend`
	* Social: `CGEncRandomAdultFriend`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillEtiquetteStreet=1.0x1`
	* Condition: `TUpIsLeader=1.0x1`

* **Report Poor Security to Ayotimiwa** `CGEncTestE`

	* Condition: `ItmBackpack03=1.0x1`
	* Social: `CGEncHomeworldManagerContact`
	* Condition: `TUpAge=1.0x1`
	* Condition: `TUpSkillAdmin=1.0x1`
	* Condition: `TUpIsLoyal=1.0x1`
